In [3]:
pip install folium


   ---------------------------------------- 0.0/113.4 kB ? eta -:--:--
   ---------------------------------------- 0.0/113.4 kB ? eta -:--:--
   --- ------------------------------------ 10.2/113.4 kB ? eta -:--:--
   --- ------------------------------------ 10.2/113.4 kB ? eta -:--:--
   --- ------------------------------------ 10.2/113.4 kB ? eta -:--:--
   ---------- ---------------------------- 30.7/113.4 kB 119.1 kB/s eta 0:00:01
   -------------- ------------------------ 41.0/113.4 kB 151.3 kB/s eta 0:00:01
   --------------------- ----------------- 61.4/113.4 kB 218.8 kB/s eta 0:00:01
   ------------------------ -------------- 71.7/113.4 kB 218.6 kB/s eta 0:00:01
   ------------------------------- ------- 92.2/113.4 kB 238.1 kB/s eta 0:00:01
   ------------------------------- ------- 92.2/113.4 kB 238.1 kB/s eta 0:00:01
   -------------------------------------- 113.4/113.4 kB 227.7 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd
import numpy as np
import folium
from folium.plugins import HeatMap
from sklearn.cluster import KMeans


In [7]:
data = pd.DataFrame({
    'latitude': [28.6139, 28.6200, 28.6350, 28.6400, 28.6500, 28.6700],
    'longitude': [77.2090, 77.2100, 77.2200, 77.2300, 77.2400, 77.3000],
    'needy_population': [200, 150, 400, 300, 500, 250]
})

In [9]:
X = data[['latitude', 'longitude']].values
weights = data['needy_population'].values


In [11]:
kmeans = KMeans(n_clusters=2, random_state=42)
kmeans.fit(X, sample_weight=weights)
data['cluster'] = kmeans.labels_

C:\Users\divya\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


In [23]:
map_center = [data['latitude'].mean(), data['longitude'].mean()]
m = folium.Map(location=map_center, zoom_start=12,title='map')

In [25]:
heat_data = [[row['latitude'], row['longitude'], row['needy_population']] for index, row in data.iterrows()]
HeatMap(heat_data, radius=25).add_to(m)

In [34]:
# Sum needy population for each cluster
cluster_summary = data.groupby('cluster')['needy_population'].sum()
# Convert to dict for easy lookup
cluster_needy = cluster_summary.to_dict()


In [42]:
# Sorting clusters by total needy population (highest first)
sorted_clusters = sorted(cluster_needy.items(), key=lambda x: x[1], reverse=True)
# Assigning colors: most needy = red,green,blue,orange.
cluster_colors = {}
colors_list = ['red', 'green', 'blue', 'orange']  
for i, (cluster, _) in enumerate(sorted_clusters):
    cluster_colors[cluster] = colors_list[i]


In [44]:
colors = ['blue', 'green', 'purple', 'orange']

for i, row in data.iterrows():
    cluster_idx = int(row['cluster'])  # <-- casting to int
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=7,
        color=colors[cluster_idx],
        fill=True,
        fill_color=colors[cluster_idx],
        fill_opacity=0.7,
        popup=f"Needy: {row['needy_population']} | Cluster: {cluster_idx}"
    ).add_to(m)


In [46]:
m